In [2]:
import pandas as pd
import numpy as np
import pyarrow as pa

In [3]:
# Load the saved parquet data
parquet_df = pd.read_parquet('../data/processed/messages.parquet')
print(parquet_df.head())

     ID                                         Clean_Text  \
0  7403  BARDEFU 2 IN 1 Multi purpose juicer ኳሊቲ የሆነ የጁ...   
1  7401  portable electrical water dispenser ባለ 3 press...   
2  7399  GROOMING SET ሶስት በአንድ የያዘ የፀጉር ማሽን እና ሼቨር የሚሰራ...   
3  7395  GROOMING SET ሶስት በአንድ የያዘ የፀጉር ማሽን እና ሼቨር የሚሰራ...   
4  7393  1L Water Bottle High Quality 1L water time sca...   

                                              Tokens  \
0  [BARDEFU, 2, IN, 1, Multi, purpose, juicer, ኳሊ...   
1  [portable, electrical, water, dispenser, ባለ, 3...   
2  [GROOMING, SET, ሶስት, በአንድ, የያዘ, የፀጉር, ማሽን, እና,...   
3  [GROOMING, SET, ሶስት, በአንድ, የያዘ, የፀጉር, ማሽን, እና,...   
4  [1L, Water, Bottle, High, Quality, 1L, water, ...   

                                           Processed  \
0  [BARDEFU, 2, IN, 1, Multi, purpose, juicer, ኳሊ...   
1  [portable, electrical, water, dispenser, ባለ, 3...   
2  [GROOMING, SET, ሶስት, በአንድ, የያዘ, የፀጉር, ማሽን, እና,...   
3  [GROOMING, SET, ሶስት, በአንድ, የያዘ, የፀጉር, ማሽን, እና,...   
4  [1L, Wa

In [ ]:

# Select 30–50 messages to label
subset = parquet_df[['ID', 'Processed']].dropna().head(30)

# Convert stringified token lists (if necessary)
if isinstance(subset.iloc[0]['Processed'], str):
    import ast
    subset['Processed'] = subset['Processed'].apply(ast.literal_eval)

# Labeling function
def label_tokens(tokens):
    print("\nInstructions:")
    print(" - Use: B-Product, I-Product, B-LOC, I-LOC, B-PRICE, I-PRICE, O")
    print(" - Press Enter to assign 'O' (Outside) to a token\n")
    
    labeled_tokens = []
    for tok in tokens:
        label = input(f"{tok}: ").strip()
        if label == "":
            label = "O"
        labeled_tokens.append((tok, label))
    return labeled_tokens

# Save labeled data to CoNLL format
with open("../data/processed/amharic_ner_conll.txt", "w", encoding="utf-8") as f:
    for _, row in subset.iterrows():
        tokens = row['Processed']
        labeled = label_tokens(tokens)
        for token, tag in labeled:
            f.write(f"{token} {tag}\n")
        f.write("\n")  # Blank line separates messages

print("✅ Done! Saved 50 labeled messages to 'amharic_ner_conll.txt'")



Instructions:
 - Use: B-Product, I-Product, B-LOC, I-LOC, B-PRICE, I-PRICE, O
 - Press Enter to assign 'O' (Outside) to a token


Instructions:
 - Use: B-Product, I-Product, B-LOC, I-LOC, B-PRICE, I-PRICE, O
 - Press Enter to assign 'O' (Outside) to a token



In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

subset = parquet_df[['ID', 'Processed']].dropna().head(30)  

labels_list = ['B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE', 'O','B-CONTACT','I-CONTACT']

# File path to save labeled data
output_path = '../data/processed/amharic_ner_conll_labeled.txt'

def label_message(tokens, idx=0):
    dropdowns = []
    
    # Create dropdowns for each token
    for tok in tokens:
        dd = widgets.Dropdown(options=labels_list, value='O', description=tok)
        dropdowns.append(dd)
    
    btn = widgets.Button(description="Submit Labels")
    out = widgets.Output()
    
    def on_submit(b):
        with out:
            clear_output()
            labels = [dd.value for dd in dropdowns]
            for token, label in zip(tokens, labels):
                print(f"{token} : {label}")

            # Save current labeled tokens to file (append mode)
            with open(output_path, 'a', encoding='utf-8') as f:
                for token, label in zip(tokens, labels):
                    f.write(f"{token} {label}\n")
                f.write('\n')  # blank line to separate messages
            
            print(f"\nLabeled message {idx+1}/{len(subset)} saved!")
            
            # Move to next message or finish
            if idx + 1 < len(subset):
                clear_output(wait=True)
                label_message(subset.iloc[idx+1]['Processed'], idx + 1)
            else:
                print("\n✅ All messages labeled and saved.")
    
    btn.on_click(on_submit)
    
    display(*dropdowns, btn, out)

# Clear the output file before starting labeling
with open(output_path, 'w', encoding='utf-8') as f:
    pass

# Start labeling from first message
label_message(subset.iloc[0]['Processed'])


Dropdown(description='BARDEFU', index=6, options=('B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PR…

Dropdown(description='2', index=6, options=('B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE', …

Dropdown(description='IN', index=6, options=('B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE',…

Dropdown(description='1', index=6, options=('B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE', …

Dropdown(description='Multi', index=6, options=('B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRIC…

...

Dropdown(description='የቴሌግራም', index=6, options=('B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRI…

Dropdown(description='ገፃችን', index=6, options=('B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE…

Dropdown(description='httpstmeShageronlinestore', index=6, options=('B-Product', 'I-Product', 'B-LOC', 'I-LOC'…

Button(description='Submit Labels', style=ButtonStyle())

Output()